### 1. General info of dataset GSE162454

This is the Jupyter Notebook for dataset GSE162454. Its dataset includes barcodes/features/matrix files for each sample.

Thus, we need to simply incorparate these barcodes/genes/matrix files and generate an AnnData object for each sample. In total, there are 3/6 pediatric samples (OS_1, OS_5, OS_6).

<span style="color:green">**[OS]**</span> Osteosarcoma

In [1]:
# Environment setup
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as anndata
import scipy

### 2. AnnData object of each sample

<span style="color:red">**IMPORTANT:**</span> rename files to get rid of prefixes 

1. `barcodes.tsv.gz`: cell barcodes, which go into `.obs`
2. `features.tsv.gz`: gene names, `.var`
3. `matrix.mtx.gz`: the expression matrix, `.X`

In [5]:
from pathlib import Path

# Specify directory paths
data_directory = Path('/scratch/user/s4543064/xiaohan-john-project/data/GSE162454')
write_directory = Path('/scratch/user/s4543064/xiaohan-john-project/write/GSE162454')

children_samples = ['OS_1', 'OS_5', 'OS_6']
sex_dict = {'OS_1': 'male', 'OS_5': 'male', 'OS_6': 'male'}
age_dict = {'OS_1': 16, 'OS_5': 14, 'OS_6': 13}

# Loop through all files in the directory
for sample_directory in data_directory.iterdir():
    sample_name = sample_directory.stem
    sample_h5ad = sample_name + '_uni.h5ad'

    is_paediatric = False
    for children_sample in children_samples:
        if children_sample in sample_name:
            is_paediatric = True
            patient_id = children_sample
            break
    
    if is_paediatric:
        sample = sc.read_10x_mtx(
            sample_directory,
            var_names='gene_symbols',  
            cache=False
        )
        
        # Create an observation metric info to store related features
        obs_metrics = pd.DataFrame(index=sample.obs_names) ## Get the identifiers
        obs_metrics['cancer_type'] = 'osteosarcoma'
        obs_metrics['dataset'] = 'GSE162454'
        obs_metrics['tissue'] = 'bone'
        obs_metrics['sample_barcode'] = sample_name
        obs_metrics['sex'] = sex_dict[patient_id]
        obs_metrics['age'] = age_dict[patient_id]
        obs_metrics['uni_barcode'] = obs_metrics['dataset'] + '_' + obs_metrics.index.astype(str)
       
        sample.obs = obs_metrics
        sample.obs.set_index("uni_barcode", drop=False, inplace=True)
        print(sample)

        # save the anndata object
        output_path = write_directory / sample_h5ad
        sample.write_h5ad(output_path, compression="gzip")

AnnData object with n_obs × n_vars = 9689 × 33538
    obs: 'cancer_type', 'dataset', 'tissue', 'sample_barcode', 'sex', 'age', 'uni_barcode'
    var: 'gene_ids', 'feature_types'
AnnData object with n_obs × n_vars = 8894 × 33538
    obs: 'cancer_type', 'dataset', 'tissue', 'sample_barcode', 'sex', 'age', 'uni_barcode'
    var: 'gene_ids', 'feature_types'
AnnData object with n_obs × n_vars = 10926 × 33538
    obs: 'cancer_type', 'dataset', 'tissue', 'sample_barcode', 'sex', 'age', 'uni_barcode'
    var: 'gene_ids', 'feature_types'


### 3. Confirmation of created AnnData objects

In [6]:
from pathlib import Path

# Specify directory paths
write_directory = Path('/scratch/user/s4543064/xiaohan-john-project/write/GSE162454')

# Loop through all files in the directory
for file in write_directory.iterdir():
    if "_uni.h5ad" in file.name:
        sample = anndata.read_h5ad(file)
        print(sample)

AnnData object with n_obs × n_vars = 10926 × 33538
    obs: 'cancer_type', 'dataset', 'tissue', 'sample_barcode', 'sex', 'age', 'uni_barcode'
    var: 'gene_ids', 'feature_types'
AnnData object with n_obs × n_vars = 9689 × 33538
    obs: 'cancer_type', 'dataset', 'tissue', 'sample_barcode', 'sex', 'age', 'uni_barcode'
    var: 'gene_ids', 'feature_types'
AnnData object with n_obs × n_vars = 8894 × 33538
    obs: 'cancer_type', 'dataset', 'tissue', 'sample_barcode', 'sex', 'age', 'uni_barcode'
    var: 'gene_ids', 'feature_types'


In [7]:
sample.var

,gene_ids,feature_types
MIR1302-2HG,ENSG00000243485,Gene Expression
FAM138A,ENSG00000237613,Gene Expression
OR4F5,ENSG00000186092,Gene Expression
AL627309.1,ENSG00000238009,Gene Expression
AL627309.3,ENSG00000239945,Gene Expression
...,...,...
AC233755.2,ENSG00000277856,Gene Expression
AC233755.1,ENSG00000275063,Gene Expression
AC240274.1,ENSG00000271254,Gene Expression
AC213203.1,ENSG00000277475,Gene Expression


In [8]:
sample.obs

,cancer_type,dataset,tissue,sample_barcode,sex,age,uni_barcode
uni_barcode,,,,,,,
GSE162454_AAACCCAAGAGCAGTC-1,osteosarcoma,GSE162454,bone,GSM4952363_OS_1,male,16,GSE162454_AAACCCAAGAGCAGTC-1
GSE162454_AAACCCAAGATAGCTA-1,osteosarcoma,GSE162454,bone,GSM4952363_OS_1,male,16,GSE162454_AAACCCAAGATAGCTA-1
GSE162454_AAACCCAAGGCCACCT-1,osteosarcoma,GSE162454,bone,GSM4952363_OS_1,male,16,GSE162454_AAACCCAAGGCCACCT-1
GSE162454_AAACCCAGTAATTAGG-1,osteosarcoma,GSE162454,bone,GSM4952363_OS_1,male,16,GSE162454_AAACCCAGTAATTAGG-1
GSE162454_AAACCCAGTGACAGCA-1,osteosarcoma,GSE162454,bone,GSM4952363_OS_1,male,16,GSE162454_AAACCCAGTGACAGCA-1
...,...,...,...,...,...,...,...
GSE162454_TTTGTTGCACGCGCAT-1,osteosarcoma,GSE162454,bone,GSM4952363_OS_1,male,16,GSE162454_TTTGTTGCACGCGCAT-1
GSE162454_TTTGTTGCATAGGCGA-1,osteosarcoma,GSE162454,bone,GSM4952363_OS_1,male,16,GSE162454_TTTGTTGCATAGGCGA-1
GSE162454_TTTGTTGCATGACAAA-1,osteosarcoma,GSE162454,bone,GSM4952363_OS_1,male,16,GSE162454_TTTGTTGCATGACAAA-1


### 4. Convert AnnData objects to SingleCellExperiment objects

In [9]:
from pathlib import Path

import anndata2ri
import rpy2.robjects as robjects
from rpy2.robjects import r
from rpy2.robjects.conversion import localconverter

# Specify directory paths
write_directory = Path('/scratch/user/s4543064/xiaohan-john-project/write/GSE162454')

# Loop through all files in the directory
for file in write_directory.iterdir():
    sample_name = file.stem
    if "_uni.h5ad" in file.name:
        sample_anndata = anndata.read_h5ad(file)
        sample_sce_file = sample_name + ".rds"

        with localconverter(anndata2ri.converter):
            sample_sce = anndata2ri.py2rpy(sample_anndata)
        
        # Save the sce object in .rds file
        robjects.globalenv["sample_sce"] = sample_sce
        sample_sce_path = write_directory / sample_sce_file
        robjects.r("saveRDS(sample_sce, file='{}')".format(sample_sce_path))

In [11]:
print(sample_sce)

class: SingleCellExperiment 
dim: 33538 8894 
metadata(0):
assays(1): X
rownames(33538): MIR1302-2HG FAM138A ... AC213203.1 FAM231C
rowData names(2): gene_ids feature_types
colnames(8894): GSE162454_AAACCCAAGAGCAGTC-1
  GSE162454_AAACCCAAGATAGCTA-1 ... GSE162454_TTTGTTGGTCTGTGTA-1
  GSE162454_TTTGTTGGTTGTTGTG-1
colData names(7): cancer_type dataset ... age uni_barcode
reducedDimNames(0):
mainExpName: NULL
altExpNames(0):

